# ASR Icefall Examples

This notebook demonstrates integrating Icefall with ART's speech recognition module.

Icefall contains speech recognition recipes for K2-FSA. Repository link: https://github.com/k2-fsa/icefall

---


## 1. Preliminaries

In [1]:
import os

import torch
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
from art.attacks.evasion.imperceptible_asr.imperceptible_asr_pytorch import ImperceptibleASRPyTorch
from art import config
from art.utils import get_file


# Set seed
np.random.seed(1234)

/home/xli257/miniconda3/envs/slu_icefall/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Audio Data

### 2.1 Download Data

In [2]:
config.ART_DATA_PATH = "~/slu"

In [3]:
# We assume that the data has been downloaded to data_dir
data_dir = os.path.join(config.ART_DATA_PATH, "fluent_speech_commands_dataset")
current_dir = %pwd

icefall_dir = '/home/xli257/slu/icefall_st/egs/slu/'

%cd $icefall_dir

/home/xli257/slu/icefall_st/egs/slu


### 2.2 Create Model and Data Utilities

In [4]:
from art.estimators.speech_recognition.pytorch_icefall import PyTorchIcefall

speech_recognizer = PyTorchIcefall()